<a href="https://colab.research.google.com/github/Matt-Ralph-Lee/machine-learning/blob/main/tensorflow/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import tensorflow as tf
import tensorflow.keras as kl
print("TensorFlow version:", tf.__version__)

from tensorflow.keras import layers
from tensorflow.keras import Model

TensorFlow version: 2.15.0


In [125]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [126]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [127]:
class Block(Model):
  def __init__(self, in_channel, out_channel, down_sample=False):
    super().__init__()
    self.act = layers.ReLU()

    if down_sample:
      stride = 2
      self.downsample = kl.Sequential([
          layers.Conv2D(out_channel, kernel_size=1, strides=(2, 2), padding='same', use_bias=False),
          layers.BatchNormalization()
      ])
    else:
      stride = 1
      self.downsample = kl.Sequential([
          layers.Conv2D(out_channel, kernel_size=1, strides=(1, 1), padding='same', use_bias=False),
          layers.BatchNormalization()
      ])

    self.conv1 = layers.Conv2D(in_channel, kernel_size=3, strides=(stride, stride), padding='same', use_bias=False)
    self.bn1 = layers.BatchNormalization()

    self.conv2 = layers.Conv2D(out_channel, kernel_size=3, strides=(1, 1), padding='same', use_bias=False)
    self.bn2 = layers.BatchNormalization()

    self.add = layers.Add()

    self.dropout = layers.Dropout(0.3)

  def call(self, x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act(x)

    x = self.dropout(x)

    x = self.conv2(x)
    x = self.bn2(x)

    x = self.dropout(x)

    if self.downsample is not None:
      identity = self.downsample(identity)

    x = self.dropout(x)

    x = self.add([identity, x])

    x = self.act(x)

    return x

In [128]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = layers.Conv2D(64, kernel_size=4, strides=(1, 1), padding='same', use_bias=False)
    self.bn = layers.BatchNormalization()
    self.act = layers.ReLU()
    self.pool1 = layers.MaxPool2D(pool_size=(3, 3),  strides=(2, 2), padding='same')
    self.pool2 = layers.GlobalAveragePooling2D()
    self.fc = layers.Dense(10, activation='softmax')

    self.layer1 = [
        Block(64, 64) for _ in range(3)
    ]

    self.layer2 = [
        Block(128, 128, False)
    ]
    for i in range(4):
      self.layer2.append(Block(128, 128))

    self.layer3 = [
        Block(128, 256, True)
    ]
    for _ in range(5):
      self.layer3.append(Block(256, 256))


    self.layer4 = [
        Block(256, 512, False)
    ]
    for _ in range(3):
      self.layer4.append(Block(512, 512))

  def call(self, x):
    x = self.conv1(x)
    x = self.bn(x)
    x = self.act(x)

    x = self.pool1(x)

    for i in range(len(self.layer1)):
      x = self.layer1[i](x)

    for i in range(len(self.layer2)):
      x = self.layer2[i](x)

    for i in range(len(self.layer3)):
      x = self.layer3[i](x)

    for i in range(len(self.layer4)):
      x = self.layer4[i](x)

    x = self.pool2(x)

    x = self.fc(x)

    return x


In [129]:
model = MyModel()

In [130]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = tf.keras.optimizers.Adam()

In [131]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [132]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [133]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [134]:
result = {}

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )
  result[epoch] = {"loss": train_loss.result(), "acc": train_accuracy.result(), "test_loss": test_loss.result(), "test_accuracy": test_accuracy.result()}

  if epoch >= 40:
    optimizer.learning_rate = 0.00001
  elif epoch >= 25:
    optimizer.learning_rate = 0.0001


Epoch 1, Loss: 1.7978787422180176, Accuracy: 31.22599983215332, Test Loss: 2.054360866546631, Test Accuracy: 34.369998931884766
